In [1]:
import numpy as np
import pandas as pd
import scipy

In [5]:
test_rating = pd.io.parsers.read_csv('raw/rating_test.csv')

In [ ]:
collaborative = pd.io.parsers.read_csv('collaborative.csv')
content = pd.io.parsers.read_csv('content.csv')

In [ ]:
user_ids = list(test_rating['user_id'].unique())
user_results = pd.DataFrame()

for user_id in user_ids:
    # Get the results of the collaborative filtering recommendations
    recommendations_cf = collaborative[collaborative['user_id']==user_id].tolist()
    recommendations_cf = set(recommendations_cf)
    
    # Get the results of the content based recommendations
    recommendations_cb = content[content['user_id']==user_id].tolist()
    recommendations_cb = set(recommendations_cb)
    
    # Get the test sample for the user
    watched_animes = test_rating['anime_id'][test_rating['user_id']].tolist()
    watched_animes = set(watched_animes)
    
    union = recommendations_cf.union(recommendations_cb)
    inter = recommendations_cf.intersection(recommendations_cb)
    
    # Get the denominator for the collaborative filtering and content based recommendations
    if len(watched_animes) > 10:
        denom = 10
    else:
        denom = len(watched_animes)
        
    # Get the 4 results for each user: number of matches for cf, cb, union and intersection
    res_cf = len(recommendations_cf.intersection(watched_animes)) * 1.0 / denom
    res_cb = len(recommendations_cb.intersection(watched_animes)) * 1.0 / denom
    res_union = len(union.intersection(watched_animes)) * 1.0 / min(len(union), len(watched_animes))
    res_intersection = len(inter.intersection(watched_animes)) * 1.0 / min(len(inter), len(watched_animes))

In [13]:
abc = pd.DataFrame(index=["user_id", "cf", "cb", "union", "intersection"])

In [14]:
abc.columns

Index([], dtype='object')